In [13]:
import pandas  as pd
import numpy as np
np.random.seed(23)

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, f_classif

from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

titanic_df = pd.read_csv('https://raw.githubusercontent.com/Kritsana135/Ml-Final/main/titanic.csv')

In [14]:
titanic_df.shape

(891, 12)

In [15]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data Exploration
    ทำการตรวจสอบข้อมูลก่อนว่ามีลักษณะเป็นอย่างไร มี missing value หรือเปล่า 
    ซึ่งจากการตรวจสอบพบว่ามี Columns
        - Cabin
        - Embarked
        - Age

    มีสอง column ที่ไม่ใช่ตัวเลข คือ Sex และ Embarked

In [16]:
total = titanic_df.isnull().sum().sort_values(ascending=False)
print(total)

Cabin          687
Age            177
Embarked         2
Fare             0
Ticket           0
Parch            0
SibSp            0
Sex              0
Name             0
Pclass           0
Survived         0
PassengerId      0
dtype: int64


# Data Cleaning
    จะทำการ ดรอปข้อมูลที่คิดว่าไม่เกี่ยวข้อง คือ ***PassengerId***, ***Name*** **Ticket** และ ***Cabin*** 
สำหรับ แอทริบิวต์ Cabin(ห้องโดยสาร) นั้นน่าจะมีความสอดคล้องกับแอทริบิวต์ Fare(ค่าโดยสาร) และเนื่องจาก Cabin มี missing value ที่มากเกินไปจึงเลือกใช้  แอทริบิวต์ Fare ซึ่งมี missing value น้อยกว่า

In [17]:
drop_columns = ['Cabin', 'PassengerId', 'Ticket', 'Name']
titanic_df = titanic_df.drop(drop_columns, axis=1)

หลังจากนั้นเราจะเติม missing value ใน Columns ต่อไปนี้ ด้วยข้อมูลที่เหลืออยู่ในแต่ละ Column
* Age ใช้ Mean เพราะข้อมูลเป็นแบบ Interval/Ratio (skewed เล็กน้อย)
* Embarked ใช้ Mode เพราะข้อมูลเป็นแบบ Nominal

In [18]:
# Fill empty values in age column
titanic_df['Age'].fillna(titanic_df['Age'].median(), inplace=True)
# df_test['Age'].fillna(df_test['Age'].median(), inplace=True)

# Fill empty data in embarked column
titanic_df['Embarked'].fillna(titanic_df['Embarked'].mode()[0], inplace=True)
# df_test['Embarked'].fillna(df_test['Embarked'].mode()[0], inplace=True)

เพื่อที่จะใช้ค่าที่ไม่ใช่ตัวเลขใน knn เราต้องแปลงมาเป็นตัวเลขก่อน

In [19]:
# change non-numerical value to numerical values
titanic_df['Sex'] = titanic_df['Sex'].map({'female': 0, 'male': 1}).astype(int)

embarked_dummies = pd.get_dummies(titanic_df['Embarked'] ,prefix="E")
titanic_df = pd.concat([titanic_df, embarked_dummies], axis=1)

#drop Embarked
titanic_df = titanic_df.drop(['Embarked'], axis=1)

titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,E_C,E_Q,E_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


Split Data

In [20]:
#split data to train and test
selected_feature = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'E_C', 'E_Q', 'E_S']
target = 'Survived'

y = titanic_df[target]
X = titanic_df[selected_feature]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=23)

Train && Test (Defalut parameter)
- ไม่ Scale  Accuracy =  0.8071748878923767

In [21]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
score = xgb.score(X_test, y_test)
print("Accuracy = ",score)

Accuracy =  0.8071748878923767


Hyperparameter Tuning

In [22]:
def tune():
  # Create the parameter grid: gbm_param_grid 
  gbm_param_grid = {
      'n_estimators': range(8, 20),
      'max_depth': range(6, 10),
      'learning_rate': [.4, .45, .5, .55, .6],
      'colsample_bytree': [.6, .7, .8, .9, 1]
  }

  # Instantiate the regressor: gbm
  gbm = XGBClassifier(n_estimators=10)

  # Perform random search: grid_mse
  xgb_random = RandomizedSearchCV(param_distributions=gbm_param_grid, 
                                      estimator = gbm, scoring = "accuracy", 
                                      verbose = 1, n_iter = 50, cv = 4)


  # Fit randomized_mse to the data
  xgb_random.fit(X_train, y_train)

  # Print the best parameters and lowest RMSE
  print("Best parameters found: ", xgb_random.best_params_)
  print("Best accuracy found: ", xgb_random.best_score_)

In [23]:
xgb = XGBClassifier(n_estimators=17,max_depth=7,learning_rate=0.4,colsample_bytree=0.7)
xgb.fit(X_train, y_train)
score = xgb.score(X_test, y_test)
print("Accuracy = ",score)

Accuracy =  0.8251121076233184


In [24]:
xgb

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.4, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=17, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

# Summary
Accuracy =  0.8251121076233184